In [1]:
import pandas as pd
import numpy as np
#-----------------------------------------Scikit----------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
#------------------------------------import feature selection----------------------------------
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2            #score_func, need to normalize to remove negative to use
from sklearn.feature_selection import f_classif
#-----------------------------------------Seaborn----------------------------------------------
import matplotlib.pyplot as plt
import ssl
import seaborn as sns
ssl._create_default_https_context = ssl._create_unverified_context
sns.set(style='darkgrid')

In [2]:
og_df = pd.read_csv('./Data/features_30_sec.csv')          #read file
song_test = pd.read_csv('./data.csv')

In [3]:
#------------------------original dataset---------------------------------------------------------
df = og_df
df = df.drop(['length','filename'],axis=1)             #remove the length and filename column since irrelevant  

song_test = song_test.drop('filename',axis=1)

In [4]:
#-----------------------------------------Encode The Genre Into Numbers-----------------------------------------
labelEncoder = LabelEncoder()                              #store encoded labels into variable
le = labelEncoder.fit(df['label'])                         #fit label into the variable
print(dict(zip(le.classes_,le.transform(le.classes_))))
df['label'] = le.transform(df['label'])                    #transform label values into numbers
Y_genre = df['label']                                      #Assign label to Y_genre
X_features = df.drop('label',axis=1)                       #Assign all features to X_feature

{'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}


In [5]:
#---------------------------------------Normalize the features between 0-1---------------------------------------
# scalar = MinMaxScaler()                                    
# scalar.fit(X_features)                                     #Fit features into scalar
# X_cols = X_features.columns
# X_features[X_cols] = scalar.transform(X_features)          #Transform features into 0-1

# scalar2 = MinMaxScaler()
# scalar2.fit(song_test)
# song_cols = song_test.columns
# song_test[song_cols] = scalar2.transform(song_test)

In [6]:
X_features.head()

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,3805.839606,9.015054e+05,...,0.752740,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035
1,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,3550.522098,2.977893e+06,...,0.927998,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282
2,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,3042.260232,7.840345e+05,...,2.451690,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025
3,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,2184.745799,1.493194e+06,...,0.780874,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339
4,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,3579.757627,1.572978e+06,...,-4.520576,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160


In [7]:
song_test.head()

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc11_mean,mfcc12_mean,mfcc13_mean,mfcc14_mean,mfcc15_mean,mfcc16_mean,mfcc17_mean,mfcc18_mean,mfcc19_mean,mfcc20_mean
0,0.224724,0.090541,0.057643,0.000962,1012.189288,78546.192988,1478.873998,63770.355500,2023.104433,4.790546e+05,...,-6.04373,-1.274762,-3.097608,-2.382829,-6.011618,1.030645,-4.220645,-4.112708,-0.109445,-3.680609
1,0.351796,0.089285,0.069279,0.002090,2380.104195,353139.223339,2515.819399,113545.117401,5217.010120,1.496111e+06,...,0.75702,2.982518,-5.925282,2.362223,-2.874656,1.177127,0.110401,1.666531,0.413473,6.651991


In [8]:
# # #------------------------------------------Feature Reduction-----------------------------------------------------
# best_feat = SelectKBest(score_func= f_classif, k=7)          #Auto select 4 features that best differentiate
# fit = best_feat.fit(X_features,Y_genre)                      #the genres

In [9]:
# feat_scores = pd.DataFrame(fit.scores_)                      #Extract optimum score of each feature
# feat_columns = pd.DataFrame(X_features.columns)              #Extract the column names

In [10]:
# sel_scores = pd.concat([feat_columns,feat_scores],axis=1)    #Concatenate the two
# sel_scores.columns = ['Features','Scores']                   #Name the two new columns

In [11]:
# sel_scores.sort_values(by=['Scores'],ascending=False)        #Sort the scores so that highest is displayed first
# sel_largest = sel_scores.nlargest(7,'Scores')                #Display the 4 best ones
# sel_largest

In [12]:
# X_features = X_features[sel_largest['Features'].T]           #Reduce the features to the 4 with best scores

In [13]:
X_features.head()

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,3805.839606,9.015054e+05,...,0.752740,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035
1,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,3550.522098,2.977893e+06,...,0.927998,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282
2,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,3042.260232,7.840345e+05,...,2.451690,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025
3,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,2184.745799,1.493194e+06,...,0.780874,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339
4,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,3579.757627,1.572978e+06,...,-4.520576,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160


In [14]:
# for feature in song_test.columns:
#     if feature not in sel_largest.values:
#         song_test = song_test.drop(feature,axis=1)
# sel_largest['Features'].values 
# song_test = song_test.reindex(columns = sel_largest['Features'].values)

In [15]:
song_test.head()

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc11_mean,mfcc12_mean,mfcc13_mean,mfcc14_mean,mfcc15_mean,mfcc16_mean,mfcc17_mean,mfcc18_mean,mfcc19_mean,mfcc20_mean
0,0.224724,0.090541,0.057643,0.000962,1012.189288,78546.192988,1478.873998,63770.355500,2023.104433,4.790546e+05,...,-6.04373,-1.274762,-3.097608,-2.382829,-6.011618,1.030645,-4.220645,-4.112708,-0.109445,-3.680609
1,0.351796,0.089285,0.069279,0.002090,2380.104195,353139.223339,2515.819399,113545.117401,5217.010120,1.496111e+06,...,0.75702,2.982518,-5.925282,2.362223,-2.874656,1.177127,0.110401,1.666531,0.413473,6.651991


In [16]:
#-------------------------------------Split data into train and test----------------------------------------
total = pd.concat([X_features,Y_genre],axis=1)
total = total.sample(frac=1)                                #randomize rows of dataset
row,col = total.shape  #extract size of dataset
split = 0.75                                          #3/4 training split
X_train = total.iloc[:int(row*split),:-1]             #obtain 75% of test data
y_train = total.iloc[:int(row*split),-1:]            #obtain 75% of genre data
X_test = total.iloc[int(row*split):,:-1]              #obtain 25% of test data
y_test = total.iloc[int(row*split):,-1:]             #obtain 25% of genre data

In [17]:
X_train.head()

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
343,0.420189,0.085688,0.173718,0.002436,2874.940335,1.276893e+06,2760.083607,537880.023283,6020.004124,5.897149e+06,...,2.205882,47.506004,-1.851751,45.939976,-6.817282,68.281509,-3.200009,59.086166,-3.548629,58.379417
572,0.250492,0.090480,0.110509,0.003491,1336.153705,7.990730e+04,1554.249504,37204.779247,2751.820122,4.866422e+05,...,-1.451717,114.914856,0.785720,65.228500,-0.403668,65.393135,-2.340947,74.813179,-2.012356,70.788467
133,0.304065,0.089344,0.012359,0.000187,1128.252612,1.248008e+05,1375.763705,82349.862182,2165.718974,6.349831e+05,...,-2.401081,90.259148,-1.891969,89.254494,-0.307660,63.502186,3.411614,70.433998,3.134486,84.851761
730,0.357656,0.084707,0.203244,0.006338,3070.608038,6.090282e+05,2975.765840,125019.562186,6653.027004,2.280632e+06,...,-3.443541,48.311687,0.395815,46.465313,-0.856712,38.450508,0.925505,35.874596,1.662314,42.696121
0,0.350088,0.088757,0.130228,0.002827,1784.165850,1.297741e+05,2002.449060,85882.761315,3805.839606,9.015054e+05,...,0.752740,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035


In [18]:
#-----------------------------------------Import algorithms------------------------------------------------------
from sklearn.ensemble import RandomForestClassifier

In [19]:
clf = RandomForestClassifier(n_estimators=180, max_depth=None,min_samples_split=4,min_samples_leaf=2,
                             max_features='auto')
clf.fit(X_train,y_train.values.ravel())

RandomForestClassifier(min_samples_leaf=2, min_samples_split=4,
                       n_estimators=180)

In [20]:
#clf.predict(song_test)

In [21]:
clf.score(X_test,y_test)

0.74

In [22]:
# from sklearn.model_selection import GridSearchCV
# n_estimators = np.arange(50,200,10)
# max_features = ['auto','sqrt']
# max_depth = [None]
# min_samples_leaf = [1,2,3]
# min_samples_split = [2,4]

# parameters = {'n_estimators':n_estimators,
#               'max_features':max_features,
#               'min_samples_leaf':min_samples_leaf,
#               'min_samples_split':min_samples_split,
#               'max_depth':max_depth}

In [23]:
# clfGridSearch = RandomForestClassifier()
# rf_Grid = GridSearchCV(clfGridSearch,parameters,cv=3,verbose=3,n_jobs=5)

In [24]:
# %%time
# rf_Grid.fit(X_train,y_train.values.ravel())
# print('Best Scores:',rf_Grid.best_score_)
# print('Best Parameters:',rf_Grid.best_params_)